In [24]:
import pandas as pd

def preprocess(raw_info_path, clean_info_path):
    lots = pd.read_csv(raw_info_path)

    lots = rename_columns(lots)
    lots = fix_dimensions(lots)
    lots = fix_price(lots)
    lots = define_sold(lots)
    lots = define_year_of_sale(lots)
    lots = lots.drop(columns=['Dimensions'])
    lots = drop_rows(lots)

    lots = lots.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

    lots.to_csv(clean_info_path, index=False)

    return lots

def rename_columns(lots):
    lots = lots.rename(columns={'Artist_name': 'Artist', 'Medium': 'Technique'})
    return lots


def fix_dimensions(lots):
    lots['Height (cm)'] = lots['Dimensions'].str.extract(r'(.*?) x')
    lots['Height (cm)'] = lots['Height (cm)'].str.replace(' cm', '').str.replace(',', '.')#.astype(float)
    lots['Width (cm)'] = lots['Dimensions'].str.extract(r'x (.*) cm')
    lots['Width (cm)'] = lots['Width (cm)'].str.replace(' cm', '').str.replace(',', '.')#.astype(float)
    return lots

def fix_price(lots):
     #lots['Price'] = lots['Price'].apply(lambda x: None if 'Price not available' in x else x)

    lots['Price'] = lots['Price'].str.extract(r'R\$ (.*?) \|')
    lots['Price'] = lots['Price'].str.replace('.', '').str.replace(',', '') #.astype(float) / 100
    # price unit is the firts charactere of price when price is float
    lots['Price unit'] = lots['Price'].apply(lambda x: str(x)[0] if isinstance(x, float) else None)
    return lots

def define_sold(lots):
    # lots['Sold'] = lots['Price'].apply(lambda x: True if any(i.isdigit() for i in x) else False)
    return lots

def define_year_of_sale(lots):
    # lots['Year of sale'] = lots['Data da Pesquisa'].str.extract(r'(\d{4})')
    return lots

def drop_rows(lots):
    lots = lots[lots['Error'] == 'False']
    # lots = lots[lots['Price (BRL)'] > 0]
    lots = lots.reset_index(drop=True)

    return lots

In [25]:
lots = preprocess('../temporary-files/artsy_auctions_artworks_info.csv', '../clean-files/artsy_auctions_artworks_info.csv')